### Guide to data cleaning

The most important thing is that we're all self consistent with our classification algorithm. Here's the general principle: First, classify by occupation then classify by the employer. For instance, "CLERK" as an occupation is sufficiently general that I just look for who the employer is. But someone working in the US army as an engineer gets ranked as an engineer, not as military. Here are some sample corner cases with resolutions.

* Government employees doing research are researchers (eg NASA, Meterologist at NWS). 

* Instructors at colleges (including community colleges) are professors, not teachers. 

* The distinction between Technology and Engineers is primarily a private vs public one. Moreover, software engineers are technology. Other engineers are engineers.

* A millwright could be an engineer because they design stuff, agriculture because mills, or under technology? I went with agriculture.

* MJ's discount warehouse with occupation SALES/BUSINESS OWNER is a business owner. 

* Insurance goes into finance.

* Secretaries and other administrative staff are fairly employer agnostic; a secretary working for an agriculture company does not need a radically different skillset from a secretary working at a law firm. For that reason, secretaries and other administrative staff get wrapped in with sales and marketing as a "fairly low level white collar" type position.

* If something really isn't clear, just chuck it in na (number 0). 

* There aren't that many students, so if you notice a student chuck them in 0. 

* Note: If their occupation is, say, bank manager or hospital administrator, put them down as bank and medical respectively. This is somewhat irritating because it means if someone works at a bank and puts down "manager,' we'll sort them differently than if they put down "bank manager."

* Technicians go in technology usually, contractors in blue collar. Use your best judgment, and when in doubt default to the more general category.

* There are a surprising number of people putting their jobs down as "Philanthropy," or maybe that's just clustering. Put them down as 0 for now, and we'll come back to it.

* If the image you have is one of a government bureaucrat, put them in government. 

### How to use this program

Run the whole notebook. When you're ready to begin, open a new cell and execute the function record_state(). The state should be a state abbreviation, q should be the quarter (either 1 or 2, but by default let's use Q1). It'll start running through all the different employers, and at the very end, it will save the accumulated database and the new csv files with an additional column for industry. Things should get easier as we move along and collect more data. If you notice certain professions coming up often that have slight differentiators in occupations, you can add them to the hardcoding.

For each industry, you'll enter the number corresponding to the industry (these are in industry_dict). I found it helpful to print industry_dict directly below my record_state() cell.



In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
get_contribution = lambda state, q: pd.read_csv(
    "contribution_data/contributions_q{1}_2019_{0}.csv".format(state, q))

industries = []
with open("industries.txt", "r") as f:
    for line in f.readlines():
        industries.append(line.strip("\n").lower())

industry_dict = dict(zip(np.arange(len(industries)), industries))
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
  "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
  "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
  "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
  "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

In [3]:
with open("database.pkl", "rb+") as f:
    database = pickle.load(f)

In [117]:
def record_state(state, q = 1, start = 0, print_loop = 100):
    """ Initiates the process to record data for a particular state and quarter.
    
    Start enables you to break out of a loop and overright something.
    
    Prints the index every print_loop entries """
    matched_industry = []
    assert q in [1, 2], "Quarter must be 1 or 1."
    assert state in states, "State must be a string describing a valid state abbreviation."
    contributions = get_contribution(state, q)
    employers = list(contributions["employer"])
    occupations = list(contributions["occupation"])
    
    check = str(input("Did you check that you aren't overwriting anything?"))
    if check[0].lower() != "y": raise ValueError("Operation halted.")

    matched_industry = matched_industry[:start]
    for i in range(start, len(employers)):
        if i % print_loop == 0: print(i)
        job = str(employers[i]).lower()
        occ = str(occupations[i]).lower()
        # Hard coding to speed things up
        
        # There are a lot of these
        gen_exec_category = ["manager", "owner", "ceo", "president", "entrepreneur", "exec", "director"]
        if occ == "retired":
            matched_industry.append("retired")
            # There are some NAs that are actually retired. 
        
        elif "software" in occ:
            matched_industry.append(industry_dict[12])
        elif "engineer" in occ or "professor" in occ or "faculty" in occ:
            matched_industry.append(industry_dict[9])
        elif "retired" in occ:
            matched_industry.append(industry_dict[1])
        elif "attorney" in occ or "lawyer" in occ:
            matched_industry.append(industry_dict[2])
        elif "doctor" in occ or "surgeon" in occ:
            matched_industry.append(industry_dict[6])
        elif np.any([i in occ for i in gen_exec_category]):
            matched_industry.append(industry_dict[7])
        elif "marketing" in occ or "sales" in occ:
            matched_industry.append(industry_dict[4])
        elif "real estate" in occ:
            matched_industry.append(industry_dict[8])
        elif "construction" in occ:
            matched_industry.append(industry_dict[17])
        elif "bank" in occ or "accountant" in occ or "invest" in occ:
            matched_industry.append(industry_dict[3])
        
        elif occ in database.keys():
            matched_industry.append(database[occ])

        else:
            found_match = False
            while not found_match:
                industry = int(input("What industry is employer {0} with occupation {1}?".format(
                    job, occupations[i])))
                if industry not in range(18):
                    print("Industry out of range")
                    continue
                else:
                    found_match = True
                    industry = industry_dict[industry]

            database[occ] = industry
            matched_industry.append(industry)
    try:
        contributions["Industry"] = matched_industry
        contributions.to_csv("contribution_data_with_industry/contribution_{0}_Q{1}.csv".format(state, q))
        with open("database.pkl", "wb") as f:
            pickle.dump(database, f)
    except:
        # Making sure I don't lose 20 minutes of my life
        print(contributions)
        print(matched_industry)

In [ ]:
skipped = ["CA", "FL"]

In [92]:
with open("database.pkl", "wb") as f:
    pickle.dump(database, f)

In [77]:
# Use to see all the states ordered by the number of donors.
donors = [len(get_contribution(state, 1)) for state in states]
together = sorted(list(zip(donors, states)))

In [ ]:
record_state(states[10], 1)

Did you check that you aren't overwriting anything? y


0
100
200
300
400
500
600


What industry is employer liberty tech with occupation Registrar? 4
What industry is employer protiviti with occupation Technology Consultant? 12


700


What industry is employer ponders funeral home with occupation Mortician? 6


In [40]:
industry_dict

{0: 'na',
 1: 'retired',
 2: 'law',
 3: 'finance/accounting',
 4: 'marketing/sales',
 5: 'government/nonprofit',
 6: 'medical',
 7: 'entrepreneur/manager/executive',
 8: 'real estate',
 9: 'researcher/professor/engineer',
 10: 'agriculture',
 11: 'teacher/writer/librarian',
 12: 'technology',
 13: 'religious',
 14: 'service/retail',
 15: 'arts',
 16: 'military/police',
 17: 'blue collar'}